<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/VITRefine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install captum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import timm
from PIL import Image
from torchvision import transforms
from pathlib import Path
import seaborn as sns
import pandas as pd
from datetime import datetime
from tqdm import tqdm
import shutil

# ----------------------------
# 1. Model Architectures
# ----------------------------
class InterpretableViT(nn.Module):
    def __init__(self, dropout_rate=0.3, hidden_dim=512):
        super(InterpretableViT, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()  # Remove original classification head
        self.embed_dim = self.vit.embed_dim if hasattr(self.vit, 'embed_dim') else 768
        self.attention_layer = nn.Sequential(
            nn.Linear(self.embed_dim, 1)
        )
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embed_dim * 2),
            nn.Dropout(dropout_rate),
            nn.Linear(self.embed_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        tokens = self.vit.forward_features(x)
        cls_token = tokens[:, 0, :]
        patch_tokens = tokens[:, 1:, :]
        attn_scores = self.attention_layer(patch_tokens)
        attn_weights = torch.softmax(attn_scores, dim=1)
        weighted_patch = torch.sum(attn_weights * patch_tokens, dim=1)
        combined = torch.cat([cls_token, weighted_patch], dim=1)
        logits = self.classifier(combined)
        return logits, attn_weights

class EnhancedViT(nn.Module):
    def __init__(self, dropout_rate=0.3, hidden_dim=512):
        super(EnhancedViT, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()
        self.embed_dim = self.vit.embed_dim if hasattr(self.vit, 'embed_dim') else self.vit.head.in_features
        self.attention_layer = nn.Sequential(
            nn.Linear(self.embed_dim, 1)
        )
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embed_dim),
            nn.Dropout(dropout_rate),
            nn.Linear(self.embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        tokens = self.vit.forward_features(x)
        attn_scores = self.attention_layer(tokens)
        attn_weights = torch.softmax(attn_scores, dim=1)
        weighted_feature = torch.sum(attn_weights * tokens, dim=1)
        out = self.classifier(weighted_feature)
        return out

class ViTModified(nn.Module):
    def __init__(self):
        super(ViTModified, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_ftrs = self.vit.head.in_features
        self.vit.head = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.vit(x)

# ----------------------------
# 2. Image Processing Functions
# ----------------------------
def preprocess_image(image_path, device, normalize=True):
    if normalize:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

def get_original_image(image_path, size=(224, 224)):
    image = Image.open(image_path).convert("RGB").resize(size, Image.LANCZOS)
    return np.array(image)

# ----------------------------
# 3. Interpretability Analyzers
# ----------------------------
from captum.attr import DeepLift, IntegratedGradients, GuidedBackprop
from captum.attr import LayerGradCam  # For GradCAM in InterpretableViT

class GradCAMAnalyzer:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradcam = LayerGradCam(self.model, self.target_layer)

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        with torch.no_grad():
            output = self.model(image_tensor)
            logits = output[0] if isinstance(output, tuple) else output
            if target_class is None:
                target_class = logits.argmax().item()
        attribution = self.gradcam.attribute(image_tensor, target=target_class)
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            attr = np.mean(attr, axis=0)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

class DeepLIFTAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        # For models returning tuples (e.g. InterpretableViT)
        if isinstance(model, InterpretableViT):
            self.deep_lift = DeepLift(self._model_wrapper)
        else:
            self.deep_lift = DeepLift(self.model)

    def _model_wrapper(self, x):
        output = self.model(x)
        return output[0] if isinstance(output, tuple) else output

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        if target_class is None:
            with torch.no_grad():
                output = self._model_wrapper(image_tensor)
                target_class = output.argmax().item()
        baseline = torch.zeros_like(image_tensor).to(self.device)
        attribution = self.deep_lift.attribute(image_tensor, baselines=baseline, target=target_class)
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            attr = np.transpose(attr, (1, 2, 0))
            attr = np.sum(np.abs(attr), axis=2)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

class LRPAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        if isinstance(model, InterpretableViT):
            self.ig = IntegratedGradients(self._model_wrapper)
        else:
            self.ig = IntegratedGradients(self.model)

    def _model_wrapper(self, x):
        output = self.model(x)
        return output[0] if isinstance(output, tuple) else output

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        if target_class is None:
            with torch.no_grad():
                output = self._model_wrapper(image_tensor)
                target_class = output.argmax().item()
        baseline = torch.zeros_like(image_tensor).to(self.device)
        attribution = self.ig.attribute(image_tensor, baselines=baseline, target=target_class, n_steps=50)
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            attr = np.transpose(attr, (1, 2, 0))
            attr = np.sum(np.abs(attr), axis=2)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

class SaliencyMapAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        if isinstance(model, InterpretableViT):
            self.guided_backprop = GuidedBackprop(self._model_wrapper)
        else:
            self.guided_backprop = GuidedBackprop(self.model)

    def _model_wrapper(self, x):
        output = self.model(x)
        return output[0] if isinstance(output, tuple) else output

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        if target_class is None:
            with torch.no_grad():
                output = self._model_wrapper(image_tensor)
                target_class = output.argmax().item()
        attribution = self.guided_backprop.attribute(image_tensor, target=target_class)
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            attr = np.transpose(attr, (1, 2, 0))
            attr = np.sum(np.abs(attr), axis=2)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

# ----------------------------
# 4. Visualization & Saving Functions
# ----------------------------
def save_visualization(original_image, attribution_maps, class_name, predicted_class, confidence, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    n_panels = 1 + len(attribution_maps)
    fig, axes = plt.subplots(1, n_panels, figsize=(n_panels * 4, 4))
    # Original image
    axes[0].imshow(original_image)
    axes[0].set_title(f"Predicted: {predicted_class}\nConf: {confidence:.2f}")
    axes[0].axis('off')
    # Attribution maps
    for i, (name, attr_map) in enumerate(attribution_maps.items(), 1):
        if name == 'Saliency':
            axes[i].imshow(attr_map, cmap='hot')
            axes[i].set_title(f"{name} Map")
        else:
            heatmap = cv2.applyColorMap(np.uint8(255 * attr_map), cv2.COLORMAP_JET)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
            if original_image.shape[:2] != heatmap.shape[:2]:
                heatmap = cv2.resize(heatmap, (original_image.shape[1], original_image.shape[0]))
            overlay = cv2.addWeighted(original_image, 0.6, heatmap, 0.4, 0)
            axes[i].imshow(overlay)
            axes[i].set_title(f"{name} Overlay")
        axes[i].axis('off')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

def save_statistics_csv(stat_list, csv_path):
    df = pd.DataFrame(stat_list)
    df.to_csv(csv_path, index=False)
    print(f"Statistics saved to {csv_path}")

def generate_confusion_matrix(stats_list, save_path, class_names):
    try:
        df = pd.DataFrame(stats_list)
        if not df.empty:
            confusion = pd.crosstab(df['true_class'], df['predicted_class'],
                                    rownames=['True'], colnames=['Predicted'])
            plt.figure(figsize=(6, 4))
            sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
            plt.title("Confusion Matrix")
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {save_path}")
    except Exception as e:
        print(f"Error generating confusion matrix: {e}")

# ----------------------------
# 5. Main Dataset Processing Function
# ----------------------------
def process_dataset_with_models(dataset_path, model_paths, output_dir_base,
                                class_names=['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    main_output_dir = Path(output_dir_base) / timestamp
    main_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Main output directory: {main_output_dir}")

    for model_idx, model_path in enumerate(model_paths, 1):
        model_name = Path(model_path).stem
        print(f"\n[{model_idx}/{len(model_paths)}] Processing with model: {model_name}")
        # Attempt to load with available architectures
        model_loaded = False
        for arch in [ViTModified, InterpretableViT, EnhancedViT]:
            try:
                model = arch().to(device)
                model.load_state_dict(torch.load(model_path, map_location=device))
                print(f"  Loaded with {arch.__name__}")
                arch_used = arch.__name__
                model_loaded = True
                break
            except Exception as e:
                print(f"  Error loading with {arch.__name__}: {e}")
        if not model_loaded:
            print(f"Skipping model {model_path} as it could not be loaded.")
            continue

        # Set up output directories for this model
        model_output_dir = main_output_dir / model_name
        deeplift_dir = model_output_dir / "deeplift_visualizations"
        lrp_dir = model_output_dir / "lrp_visualizations"
        saliency_dir = model_output_dir / "saliency_visualizations"
        combined_dir = model_output_dir / "combined_visualizations"
        correct_images_dir = model_output_dir / "correct_images"
        for d in [deeplift_dir, lrp_dir, saliency_dir, combined_dir, correct_images_dir]:
            d.mkdir(parents=True, exist_ok=True)

        # Initialize analyzers
        deeplift_analyzer = DeepLIFTAnalyzer(model, device)
        lrp_analyzer = LRPAnalyzer(model, device)
        saliency_analyzer = SaliencyMapAnalyzer(model, device)
        model_stats = []

        # Process images in each class folder
        dataset_path = Path(dataset_path)
        for class_folder in dataset_path.iterdir():
            if not class_folder.is_dir():
                continue
            true_class = class_folder.name
            if true_class not in class_names:
                print(f"Skipping folder '{true_class}' as it is not in class_names")
                continue

            # Create class-specific directories under each output type
            class_deeplift_dir = deeplift_dir / true_class
            class_lrp_dir = lrp_dir / true_class
            class_saliency_dir = saliency_dir / true_class
            class_combined_dir = combined_dir / true_class
            class_correct_dir = correct_images_dir / true_class
            for d in [class_deeplift_dir, class_lrp_dir, class_saliency_dir, class_combined_dir, class_correct_dir]:
                d.mkdir(parents=True, exist_ok=True)

            # Find image files (jpg, jpeg, png)
            image_files = list(class_folder.glob("*.[jJ][pP][gG]")) + \
                          list(class_folder.glob("*.[jJ][pP][eE][gG]")) + \
                          list(class_folder.glob("*.[pP][nN][gG]"))
            print(f"Processing {len(image_files)} images in class '{true_class}'...")

            for image_path in tqdm(image_files, desc=f"[{model_name}] {true_class}"):
                try:
                    img_path_str = str(image_path)
                    image_tensor = preprocess_image(img_path_str, device)
                    original_image = get_original_image(img_path_str)
                    with torch.no_grad():
                        output = model(image_tensor)
                        logits = output[0] if isinstance(output, tuple) else output
                        probs = F.softmax(logits, dim=1)
                        predicted_class_idx = torch.argmax(probs, dim=1).item()
                        confidence = probs[0, predicted_class_idx].item()
                    predicted_class = class_names[predicted_class_idx] if predicted_class_idx < len(class_names) else str(predicted_class_idx)
                    is_correct = (predicted_class == true_class)

                    # Generate visualizations only for correctly predicted images
                    if is_correct:
                        deeplift_attr = deeplift_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                        lrp_attr = lrp_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                        # For InterpretableViT use GradCAM; otherwise saliency overlay
                        if isinstance(model, InterpretableViT):
                            target_layer = model.vit.patch_embed.proj
                            gradcam_analyzer = GradCAMAnalyzer(model, target_layer)
                            gradcam_attr = gradcam_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                            attributions = {'DeepLIFT': deeplift_attr,
                                            'LRP': lrp_attr,
                                            'GradCAM': gradcam_attr}
                        else:
                            saliency_attr = saliency_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                            attributions = {'DeepLIFT': deeplift_attr,
                                            'LRP': lrp_attr,
                                            'Saliency': saliency_attr}

                        # Save individual visualizations
                        save_visualization(original_image, {'DeepLIFT': deeplift_attr},
                                           true_class, predicted_class, confidence,
                                           str(class_deeplift_dir / f"{image_path.stem}_deeplift.png"))
                        save_visualization(original_image, {'LRP': lrp_attr},
                                           true_class, predicted_class, confidence,
                                           str(class_lrp_dir / f"{image_path.stem}_lrp.png"))
                        if 'Saliency' in attributions:
                            save_visualization(original_image, {'Saliency': saliency_attr},
                                               true_class, predicted_class, confidence,
                                               str(class_saliency_dir / f"{image_path.stem}_saliency.png"))
                        else:
                            save_visualization(original_image, {'GradCAM': gradcam_attr},
                                               true_class, predicted_class, confidence,
                                               str(class_saliency_dir / f"{image_path.stem}_gradcam.png"))
                        # Save combined visualization
                        save_visualization(original_image, attributions,
                                           true_class, predicted_class, confidence,
                                           str(class_combined_dir / f"{image_path.stem}_combined.png"))
                        # Copy original image to correct images folder
                        shutil.copy2(img_path_str, str(class_correct_dir / image_path.name))

                    # Log statistics for each image
                    model_stats.append({
                        "model": arch_used,
                        "file": img_path_str,
                        "true_class": true_class,
                        "predicted_class": predicted_class,
                        "confidence": confidence,
                        "correct": is_correct
                    })
                except Exception as e:
                    print(f"Error processing {image_path} with model {arch_used}: {e}")

        # Save per-model statistics and confusion matrix
        stats_csv_path = model_output_dir / f"{model_name}_statistics.csv"
        save_statistics_csv(model_stats, str(stats_csv_path))
        confusion_matrix_path = model_output_dir / f"{model_name}_confusion_matrix.png"
        generate_confusion_matrix(model_stats, str(confusion_matrix_path), class_names)
        df_stats = pd.DataFrame(model_stats)
        accuracy = df_stats['correct'].mean() * 100
        print(f"Model '{model_name}' (Architecture: {arch_used}) accuracy: {accuracy:.2f}%")
        with open(str(model_output_dir / f"{model_name}_summary.txt"), "w") as f:
            f.write(f"Model: {model_name} (Architecture: {arch_used})\n")
            f.write(f"Processed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Overall Accuracy: {accuracy:.2f}%\n\n")
            f.write("Per-Class Metrics:\n")
            for cls in class_names:
                cls_df = df_stats[df_stats['true_class'] == cls]
                cls_acc = cls_df['correct'].mean() * 100 if len(cls_df) > 0 else 0
                correct_count = len(cls_df[cls_df['correct']])
                total_count = len(cls_df)
                f.write(f"  {cls}: {cls_acc:.2f}% ({correct_count}/{total_count} correct)\n")
        print(f"Completed processing with model: {model_name}")

    print(f"\nAll processing complete. Results saved to {main_output_dir}")

    # Optionally, generate a combined report across models.
    try:
        all_stats = []
        for model_path in model_paths:
            m_name = Path(model_path).stem
            stats_path = main_output_dir / m_name / f"{m_name}_statistics.csv"
            if stats_path.exists():
                df_m = pd.read_csv(stats_path)
                all_stats.append(df_m)
        if all_stats:
            combined_df = pd.concat(all_stats)
            combined_stats_path = main_output_dir / "all_models_comparison.csv"
            combined_df.to_csv(combined_stats_path, index=False)
            model_summary = combined_df.groupby('model')['correct'].agg(['mean', 'count']).reset_index()
            model_summary['accuracy'] = model_summary['mean'] * 100
            model_summary = model_summary.drop('mean', axis=1)
            model_summary = model_summary.rename(columns={'count': 'total_images', 'accuracy': 'accuracy_percent'})
            model_summary = model_summary.sort_values('accuracy_percent', ascending=False)
            model_summary.to_csv(main_output_dir / "model_accuracy_comparison.csv", index=False)
            plt.figure(figsize=(10, 6))
            sns.barplot(x='model', y='accuracy_percent', data=model_summary)
            plt.title('Model Accuracy Comparison')
            plt.xlabel('Model')
            plt.ylabel('Accuracy (%)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(main_output_dir / "model_accuracy_comparison.png", dpi=300)
            plt.close()
            print(f"Generated combined model comparison reports in {main_output_dir}")
    except Exception as e:
        print(f"Error generating combined summary: {e}")

# ----------------------------
# 6. Main Execution Example
# ----------------------------
if __name__ == "__main__":
    # For Colab, you might need to mount your drive:
    from google.colab import drive
    drive.mount('/content/drive')

    # Define your dataset and output paths (update these as needed)
    DATASET_PATH = "/content/drive/My Drive/FYP/Black Background"

    # List your model checkpoint paths (update these as needed)
    MODEL_PATHS = [
        "/content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ2_20241222/latest_model_vit_20241222.pth",
        "/content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ3_20250208/final_model_vit_20250208.pth",
        "/content/drive/My Drive/FYP/MODELS/VIT/InterpretableViT_20250213/final_model_vit_20250213.pth",
    ]
    # Define class names (subfolder names)
    CLASS_NAMES = ["Glaucous_Winged_Gull", "Slaty_Backed_Gull"]
    # Base output directory
    OUTPUT_DIR_BASE = "./model_output"

    valid_models = [m for m in MODEL_PATHS if os.path.exists(m)]
    if not valid_models:
        print("ERROR: No valid models found. Please check your model paths.")
    else:
        process_dataset_with_models(DATASET_PATH, valid_models, OUTPUT_DIR_BASE, CLASS_NAMES)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Main output directory: model_output/20250310_013627

[1/3] Processing with model: latest_model_vit_20241222


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-2-c0581b4ff0b7>:287: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the

  Loaded with ViTModified
Processing 143 images in class 'Slaty_Backed_Gull'...


[latest_model_vit_20241222] Slaty_Backed_Gull:   1%|▏         | 2/143 [00:00<00:55,  2.56it/s]/usr/local/lib/python3.11/dist-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/captum/attr/_core/deep_lift.py:304: UserWarning: Setting forward, backward hooks and attributes on non-linear
               activations. The hooks and attributes will be removed
            after the attribution is finished
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(
[latest_model_vit_20241222] Slaty_Backed_Gull:  39%|███▉      | 56/143 [03:03<04:24,  3.04s/it]

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import timm
from PIL import Image
from torchvision import transforms
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import seaborn as sns

# ----------------------------
# 1. Model Architectures
# ----------------------------

# (a) InterpretableViT: Uses the [CLS] token and an attention over patch tokens.
class InterpretableViT(nn.Module):
    def __init__(self, dropout_rate=0.3, hidden_dim=512):
        """
        This model uses a pre-trained ViT backbone and removes its original classification head.
        It then:
          - Extracts the [CLS] token (which is already well-trained)
          - Computes a learned attention over patch tokens (the remaining tokens)
          - Aggregates the patch tokens via a weighted sum
          - Concatenates the CLS token and the weighted patch summary
          - Feeds the combined representation through a custom MLP classifier
        The attention weights are returned for later visualization.
        """
        super(InterpretableViT, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()  # Remove the original classification head

        # Determine embedding dimension (most ViT models have an attribute 'embed_dim')
        self.embed_dim = self.vit.embed_dim if hasattr(self.vit, 'embed_dim') else 768

        # Attention layer on patch tokens (ignoring the CLS token)
        self.attention_layer = nn.Sequential(
            nn.Linear(self.embed_dim, 1)
        )

        # Classifier head: use both the CLS token and a weighted average of patch tokens.
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embed_dim * 2),
            nn.Dropout(dropout_rate),
            nn.Linear(self.embed_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 2)
        )

    def forward(self, x):
        # Get token embeddings from ViT; output shape: (B, N+1, embed_dim)
        tokens = self.vit.forward_features(x)
        # The first token is the [CLS] token
        cls_token = tokens[:, 0, :]  # Shape: (B, embed_dim)
        # Remaining tokens are patch tokens
        patch_tokens = tokens[:, 1:, :]  # Shape: (B, N, embed_dim)
        # Compute attention scores over patch tokens
        attn_scores = self.attention_layer(patch_tokens)  # (B, N, 1)
        attn_weights = torch.softmax(attn_scores, dim=1)    # (B, N, 1)
        # Weighted average of patch tokens
        weighted_patch = torch.sum(attn_weights * patch_tokens, dim=1)  # (B, embed_dim)
        # Combine the CLS token and weighted patch representation
        combined = torch.cat([cls_token, weighted_patch], dim=1)  # (B, 2*embed_dim)
        logits = self.classifier(combined)  # (B, 2)
        return logits, attn_weights  # Return both logits and attention weights

# (b) EnhancedViT: Pools patch tokens with an attention mechanism then classifies.
class EnhancedViT(nn.Module):
    def __init__(self, dropout_rate=0.3, hidden_dim=512):
        """
        Initializes the enhanced ViT model.
          - Loads a pre-trained ViT backbone.
          - Removes the original classification head.
          - Adds an attention mechanism to pool patch tokens.
          - Adds a custom MLP classifier head.
        """
        super(EnhancedViT, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.vit.head = nn.Identity()

        # Get the embedding dimension (most timm ViT models have 'embed_dim')
        if hasattr(self.vit, 'embed_dim'):
            self.embed_dim = self.vit.embed_dim
        else:
            self.embed_dim = self.vit.head.in_features

        # Attention mechanism: compute an attention score for each token (patch)
        self.attention_layer = nn.Sequential(
            nn.Linear(self.embed_dim, 1)
        )

        # Custom classifier head
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.embed_dim),
            nn.Dropout(dropout_rate),
            nn.Linear(self.embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, 2)  # Binary classification (2 classes)
        )

    def forward(self, x):
        tokens = self.vit.forward_features(x)  # Shape: [B, num_tokens, embed_dim]
        attn_scores = self.attention_layer(tokens)  # Shape: [B, num_tokens, 1]
        attn_weights = torch.softmax(attn_scores, dim=1)
        weighted_feature = torch.sum(attn_weights * tokens, dim=1)  # Shape: [B, embed_dim]
        out = self.classifier(weighted_feature)
        return out

# (c) ViTModified: A simple modification that replaces the classification head.
class ViTModified(nn.Module):
    def __init__(self):
        super(ViTModified, self).__init__()
        self.vit = timm.create_model('vit_base_patch16_224', pretrained=True)
        num_ftrs = self.vit.head.in_features
        self.vit.head = nn.Linear(num_ftrs, 2)  # Output 2 classes

    def forward(self, x):
        return self.vit(x)

# ----------------------------
# 2. Utility Functions for Image Processing
# ----------------------------
def preprocess_image(image_path, device, normalize=True):
    """
    Loads an image and applies transformations.
    Returns a tensor of shape (1, 3, 224, 224) on the specified device.
    """
    if normalize:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ])
    else:
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
    image = Image.open(image_path).convert("RGB")
    return transform(image).unsqueeze(0).to(device)

def get_original_image(image_path, size=(224, 224)):
    """
    Loads and resizes an image without normalization for visualization.
    """
    image = Image.open(image_path).convert("RGB").resize(size, Image.LANCZOS)
    return np.array(image)

# ----------------------------
# 3. Interpretability Analyzers (DeepLIFT, LRP, Saliency)
# ----------------------------
from captum.attr import DeepLift, LayerIntegratedGradients

class DeepLIFTAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.deep_lift = DeepLift(self.model)

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        if target_class is None:
            with torch.no_grad():
                output = self.model(image_tensor)
                # Handle tuple output (e.g. from InterpretableViT)
                if isinstance(output, tuple):
                    logits = output[0]
                else:
                    logits = output
                target_class = logits.argmax().item()
        baseline = torch.zeros_like(image_tensor).to(self.device)
        attribution = self.deep_lift.attribute(
            image_tensor,
            baselines=baseline,
            target=target_class
        )
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            if attr.shape[0] == 1:
                attr = attr.squeeze(0)
            else:
                attr = np.transpose(attr, (1, 2, 0))
                if attr.shape[2] not in [1, 3]:
                    attr = np.mean(attr, axis=2)
                else:
                    attr = np.mean(np.abs(attr), axis=2)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

class LRPAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.layer_ig = LayerIntegratedGradients(self.model, self.model.vit.patch_embed)

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        if target_class is None:
            with torch.no_grad():
                output = self.model(image_tensor)
                if isinstance(output, tuple):
                    logits = output[0]
                else:
                    logits = output
                target_class = logits.argmax().item()
        attribution, _ = self.layer_ig.attribute(
            image_tensor,
            target=target_class,
            n_steps=50,
            return_convergence_delta=True
        )
        attr = attribution.detach().cpu().numpy()[0]
        if attr.ndim == 3:
            if attr.shape[0] == 1:
                attr = attr.squeeze(0)
            else:
                attr = np.transpose(attr, (1, 2, 0))
                if attr.shape[2] not in [1, 3]:
                    attr = np.mean(attr, axis=2)
                else:
                    attr = np.mean(np.abs(attr), axis=2)
        attr = (attr - attr.min()) / (attr.max() - attr.min() + 1e-8)
        return attr

class SaliencyMapAnalyzer:
    def __init__(self, model, device):
        self.model = model
        self.device = device

    def generate_attribution(self, image_tensor, target_class=None):
        self.model.eval()
        image_tensor_with_grad = image_tensor.clone().detach().requires_grad_(True)
        output = self.model(image_tensor_with_grad)
        if isinstance(output, tuple):
            logits = output[0]
        else:
            logits = output
        if target_class is None:
            target_class = logits.argmax().item()
        self.model.zero_grad()
        logits[0, target_class].backward()
        gradients = image_tensor_with_grad.grad.data
        saliency, _ = torch.max(gradients.abs(), dim=1)
        saliency = saliency.squeeze().cpu().numpy()
        saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min() + 1e-8)
        return saliency

# ----------------------------
# 4. Visualization & Saving Functions
# ----------------------------
def save_visualization(original_image, attribution_maps, class_name, predicted_class, confidence, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    n_panels = 1 + len(attribution_maps)
    fig, axes = plt.subplots(1, n_panels, figsize=(n_panels * 4, 4))
    axes[0].imshow(original_image)
    axes[0].set_title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2f}")
    axes[0].axis('off')
    for i, (name, attr_map) in enumerate(attribution_maps.items(), 1):
        if name == 'Saliency':
            axes[i].imshow(attr_map, cmap='hot')
            axes[i].set_title(f"{name} Map")
        else:
            heatmap = cv2.applyColorMap(np.uint8(255 * attr_map), cv2.COLORMAP_JET)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)
            if original_image.shape[:2] != heatmap.shape[:2]:
                original_resized = cv2.resize(original_image, (heatmap.shape[1], heatmap.shape[0]))
            else:
                original_resized = original_image
            overlay = cv2.addWeighted(original_resized, 0.6, heatmap, 0.4, 0)
            axes[i].imshow(overlay)
            axes[i].set_title(f"{name} Overlay")
        axes[i].axis('off')
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

def save_statistics_csv(stat_list, csv_path):
    df = pd.DataFrame(stat_list)
    df.to_csv(csv_path, index=False)
    print(f"Statistics saved to {csv_path}")

def generate_confusion_matrix(stats_list, save_path, class_names):
    try:
        df = pd.DataFrame(stats_list)
        if not df.empty:
            confusion = pd.crosstab(df['true_class'], df['predicted_class'],
                                    rownames=['True'], colnames=['Predicted'])
            plt.figure(figsize=(6, 4))
            sns.heatmap(confusion, annot=True, fmt="d", cmap="Blues")
            plt.title("Confusion Matrix")
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            plt.close()
            print(f"Confusion matrix saved to: {save_path}")
    except Exception as e:
        print(f"Error generating confusion matrix: {e}")

# ----------------------------
# 5. Main Processing Function with Dynamic Architecture Selection
# ----------------------------
def process_dataset_with_models(dataset_path, model_paths, output_dir_base,
                                class_names=['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    main_output_dir = Path(output_dir_base) / timestamp
    main_output_dir.mkdir(parents=True, exist_ok=True)
    print(f"Main output directory: {main_output_dir}")

    for model_idx, model_path in enumerate(model_paths, 1):
        model_name = Path(model_path).stem
        print(f"\n[{model_idx}/{len(model_paths)}] Processing with model file: {model_name}")
        # Try loading the checkpoint with each architecture until one succeeds.
        model_loaded = False
        for arch in [ViTModified, InterpretableViT, EnhancedViT]:
            try:
                model = arch().to(device)
                model.load_state_dict(torch.load(model_path, map_location=device))
                print(f"Successfully loaded checkpoint with {arch.__name__}")
                model_loaded = True
                arch_used = arch.__name__
                break
            except Exception as e:
                print(f"Error loading model from {model_path} with {arch.__name__}: {e}")
        if not model_loaded:
            print(f"Skipping model {model_path} as it could not be loaded with any known architecture.")
            continue

        # Set up output directories for this model
        model_output_dir = main_output_dir / model_name
        deeplift_dir = model_output_dir / "deeplift_visualizations"
        lrp_dir = model_output_dir / "lrp_visualizations"
        saliency_dir = model_output_dir / "saliency_visualizations"
        correct_images_dir = model_output_dir / "correct_images"
        os.makedirs(deeplift_dir, exist_ok=True)
        os.makedirs(lrp_dir, exist_ok=True)
        os.makedirs(saliency_dir, exist_ok=True)
        os.makedirs(correct_images_dir, exist_ok=True)

        # Initialize interpreters
        deeplift_analyzer = DeepLIFTAnalyzer(model, device)
        lrp_analyzer = LRPAnalyzer(model, device)
        saliency_analyzer = SaliencyMapAnalyzer(model, device)
        model_stats = []

        # Process images from each class folder
        for class_folder in Path(dataset_path).iterdir():
            if not class_folder.is_dir():
                continue
            true_class = class_folder.name
            if true_class not in class_names:
                print(f"Skipping folder '{true_class}' as it is not in class_names")
                continue

            class_deeplift_dir = deeplift_dir / true_class
            class_lrp_dir = lrp_dir / true_class
            class_saliency_dir = saliency_dir / true_class
            class_correct_dir = correct_images_dir / true_class

            os.makedirs(class_deeplift_dir, exist_ok=True)
            os.makedirs(class_lrp_dir, exist_ok=True)
            os.makedirs(class_saliency_dir, exist_ok=True)
            os.makedirs(class_correct_dir, exist_ok=True)

            image_files = list(class_folder.glob("*.[jJ][pP][gG]")) + \
                          list(class_folder.glob("*.[jJ][pP][eE][gG]")) + \
                          list(class_folder.glob("*.[pP][nN][gG]"))
            print(f"Processing {len(image_files)} images in class '{true_class}'...")

            for image_path in tqdm(image_files, desc=f"[{model_name}] Class: {true_class}"):
                try:
                    image_tensor = preprocess_image(str(image_path), device)
                    original_image = get_original_image(str(image_path))
                    with torch.no_grad():
                        output = model(image_tensor)
                        # Handle models that return a tuple (e.g., InterpretableViT)
                        if isinstance(output, tuple):
                            logits = output[0]
                        else:
                            logits = output
                        probs = F.softmax(logits, dim=1)
                        predicted_class_idx = torch.argmax(probs, dim=1).item()
                        confidence = probs[0, predicted_class_idx].item()
                    is_correct = (predicted_class_idx == class_names.index(true_class))
                    if is_correct:
                        deeplift_attr = deeplift_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                        lrp_attr = lrp_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                        saliency_attr = saliency_analyzer.generate_attribution(image_tensor, predicted_class_idx)
                        attributions = {
                            'DeepLIFT': deeplift_attr,
                            'LRP': lrp_attr,
                            'Saliency': saliency_attr
                        }
                        # Save visualizations
                        save_visualization(
                            original_image,
                            {'DeepLIFT': deeplift_attr},
                            true_class,
                            class_names[predicted_class_idx],
                            confidence,
                            str(class_deeplift_dir / f"{image_path.stem}_deeplift.png")
                        )
                        save_visualization(
                            original_image,
                            {'LRP': lrp_attr},
                            true_class,
                            class_names[predicted_class_idx],
                            confidence,
                            str(class_lrp_dir / f"{image_path.stem}_lrp.png")
                        )
                        save_visualization(
                            original_image,
                            {'Saliency': saliency_attr},
                            true_class,
                            class_names[predicted_class_idx],
                            confidence,
                            str(class_saliency_dir / f"{image_path.stem}_saliency.png")
                        )
                        combined_dir = model_output_dir / "combined_visualizations" / true_class
                        os.makedirs(combined_dir, exist_ok=True)
                        save_visualization(
                            original_image,
                            attributions,
                            true_class,
                            class_names[predicted_class_idx],
                            confidence,
                            str(combined_dir / f"{image_path.stem}_combined.png")
                        )
                        shutil.copy2(str(image_path), str(class_correct_dir / image_path.name))
                    model_stats.append({
                        "model": arch_used,
                        "file": str(image_path),
                        "true_class": true_class,
                        "predicted_class": class_names[predicted_class_idx],
                        "confidence": confidence,
                        "correct": is_correct
                    })
                except Exception as e:
                    print(f"Error processing {image_path} with model {arch_used}: {e}")

        stats_csv_path = model_output_dir / f"{model_name}_statistics.csv"
        save_statistics_csv(model_stats, str(stats_csv_path))
        confusion_matrix_path = model_output_dir / f"{model_name}_confusion_matrix.png"
        generate_confusion_matrix(model_stats, str(confusion_matrix_path), class_names)
        df = pd.DataFrame(model_stats)
        accuracy = df['correct'].mean() * 100
        print(f"Model '{model_name}' (loaded as {arch_used}) accuracy: {accuracy:.2f}%")
        with open(str(model_output_dir / f"{model_name}_summary.txt"), "w") as f:
            f.write(f"Model: {model_name} (Architecture: {arch_used})\n")
            f.write(f"Processed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Overall Accuracy: {accuracy:.2f}%\n\n")
            f.write("Per-Class Metrics:\n")
            for class_name in class_names:
                class_df = df[df['true_class'] == class_name]
                class_acc = class_df['correct'].mean() * 100 if len(class_df) > 0 else 0
                f.write(f"  {class_name}: {class_acc:.2f}% ({len(class_df[class_df['correct']])}/{len(class_df)} correct)\n")
        print(f"Completed processing with model: {model_name}")

    print(f"\nAll processing complete. Results saved to {main_output_dir}")

    # Optionally, generate a combined report across models.
    try:
        all_stats = []
        for model_path in model_paths:
            model_name = Path(model_path).stem
            stats_path = main_output_dir / model_name / f"{model_name}_statistics.csv"
            if stats_path.exists():
                df = pd.read_csv(stats_path)
                all_stats.append(df)
        if all_stats:
            combined_df = pd.concat(all_stats)
            combined_stats_path = main_output_dir / "all_models_comparison.csv"
            combined_df.to_csv(combined_stats_path, index=False)
            model_summary = combined_df.groupby('model')['correct'].agg(['mean', 'count']).reset_index()
            model_summary['accuracy'] = model_summary['mean'] * 100
            model_summary = model_summary.drop('mean', axis=1)
            model_summary = model_summary.rename(columns={'count': 'total_images', 'accuracy': 'accuracy_percent'})
            model_summary = model_summary.sort_values('accuracy_percent', ascending=False)
            model_summary.to_csv(main_output_dir / "model_accuracy_comparison.csv", index=False)
            plt.figure(figsize=(10, 6))
            sns.barplot(x='model', y='accuracy_percent', data=model_summary)
            plt.title('Model Accuracy Comparison')
            plt.xlabel('Model')
            plt.ylabel('Accuracy (%)')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(main_output_dir / "model_accuracy_comparison.png", dpi=300)
            plt.close()
            print(f"Generated combined model comparison reports in {main_output_dir}")
    except Exception as e:
        print(f"Error generating combined summary: {e}")

# ----------------------------
# 6. Google Colab Main Execution (if applicable)
# ----------------------------
# Install required packages (if needed)
!pip install timm captum

# Mount Google Drive (specific to Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Define your dataset and output paths (update these as needed)
DATASET_PATH = "/content/drive/My Drive/FYP/Black Background"  # Contains class folders
OUTPUT_BASE = "/content/drive/My Drive/FYP/Results/ViT_Interpretability"  # Base output directory
CLASS_NAMES = ["Glaucous_Winged_Gull", "Slaty_Backed_Gull"]

# List your model checkpoint paths (update these as needed)
MODEL_PATHS = [
    # "/content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ2_20241222/latest_model_vit_20241222.pth",
    "/content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ3_20250208/final_model_vit_20250208.pth",
    "/content/drive/My Drive/FYP/MODELS/VIT/InterpretableViT_20250213/final_model_vit_20250213.pth",
]

if not os.path.exists(DATASET_PATH):
    print(f"ERROR: Dataset path not found: {DATASET_PATH}")
else:
    os.makedirs(OUTPUT_BASE, exist_ok=True)
    valid_models = []
    for model_path in MODEL_PATHS:
        if os.path.exists(model_path):
            valid_models.append(model_path)
        else:
            print(f"WARNING: Model not found: {model_path}")
    if not valid_models:
        print("ERROR: No valid models found. Please check your model paths.")
    else:
        print(f"Found {len(valid_models)} valid models. Starting processing...")
        process_dataset_with_models(
            dataset_path=DATASET_PATH,
            model_paths=valid_models,
            output_dir_base=OUTPUT_BASE,
            class_names=CLASS_NAMES
        )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 2 valid models. Starting processing...
Using device: cuda
Main output directory: /content/drive/My Drive/FYP/Results/ViT_Interpretability/20250309_065739

[1/2] Processing with model file: final_model_vit_20250208


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

<ipython-input-2-38eb490eac3f>:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Error loading model from /content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ3_20250208/final_model_vit_20250208.pth with ViTModified: Error(s) in loading state_dict for ViTModified:
	Missing key(s) in state_dict: "vit.head.weight", "vit.head.bias". 
	Unexpected key(s) in state_dict: "attention_layer.0.weight", "attention_layer.0.bias", "classifier.0.weight", "classifier.0.bias", "classifier.2.weight", "classifier.2.bias", "classifier.5.weight", "classifier.5.bias". 
Error loading model from /content/drive/My Drive/FYP/MODELS/VIT/VIT2_HQ3_20250208/final_model_vit_20250208.pth with InterpretableViT: Error(s) in loading state_dict for InterpretableViT:
	size mismatch for classifier.0.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1536]).
	size mismatch for classifier.0.bias: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([1536]).
	size mismatch for classifier.2.weight: copyi

...

Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/126.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/127.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/131.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/130.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/134.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple


[final_model_vit_20250213] Class: Glaucous_Winged_Gull:  98%|█████████▊| 122/124 [00:11<00:00, 16.61it/s]

Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/133.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/132.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/135.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/137.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple
Error processing /content/drive/My Drive/FYP/Black Background/Glaucous_Winged_Gull/138.jpg with model InterpretableViT: chunk(): argument 'input' (position 1) must be Tensor, not tuple


[final_model_vit_20250213] Class: Glaucous_Winged_Gull: 100%|██████████| 124/124 [00:11<00:00, 10.34it/s]


Statistics saved to /content/drive/My Drive/FYP/Results/ViT_Interpretability/20250309_065739/final_model_vit_20250213/final_model_vit_20250213_statistics.csv
Confusion matrix saved to: /content/drive/My Drive/FYP/Results/ViT_Interpretability/20250309_065739/final_model_vit_20250213/final_model_vit_20250213_confusion_matrix.png
Model 'final_model_vit_20250213' (loaded as InterpretableViT) accuracy: 0.00%
Completed processing with model: final_model_vit_20250213

All processing complete. Results saved to /content/drive/My Drive/FYP/Results/ViT_Interpretability/20250309_065739
Generated combined model comparison reports in /content/drive/My Drive/FYP/Results/ViT_Interpretability/20250309_065739
